In [ ]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import seaborn as sns
import datetime
from group_lasso import GroupLasso
from sklearn.utils import resample, check_random_state
from sklearn.model_selection import cross_val_score, cross_validate

from extra_functions import *

#Silence some warnings (remove before final version)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('energydata_complete.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

fig = plot_data(df)
fig

### Taking an 24-hour mean

In [ ]:
df = df.resample('24h').mean()

### Generating extra features to describe time
weekday: number [0,6]\
weekstatus: binary describing weekend (1) or not (0)\
NSM: Number of Seconds from Midnight

These are used for filtering the data

In [ ]:
weekday = np.zeros(len(df))
weekstatus = np.zeros(len(df))
NSM = np.zeros(len(df))
month = np.zeros(len(df))

for i, val in enumerate(df.index):
    weekday[i] = val.weekday()
    weekstatus[i] = (weekday[i] >= 5)  # False for workday, True for weekend
    NSM[i] = (val - val.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()
    month[i] = val.month

df['weekday'] = weekday
df['week status'] = weekstatus
df['NSM'] = NSM
df['month'] = month

In [ ]:
plt.figure()
plt.scatter(df['NSM'],df['Appliances'])
plt.xlabel('NSM')
plt.ylabel('Appliances')
plt.show()

### Filtering data and making training set
Example: Only february, after 16:00 and workday

In [ ]:
#df_train = df[(df.index.month==2) & (df['NSM']>=16*3600) & (df['NSM']<24*3600)]
df_train = df[(df.index.month==2)]
df_train = df_train.drop(['weekday', 'week status','month','NSM'], axis=1) # dropping the features used for filtering

# Training data
y = np.array(df_train['Appliances']).reshape(-1,1)
X = np.array(df_train[df_train.columns[1:]])
X, y = standardize(X,y)

### Correlations of covariates

In [ ]:
cor = df_train[df_train.columns[1:]].corr()
fig, ax = plt.subplots(figsize=(10,10)) 
sns.heatmap(cor, square=True, xticklabels=True, yticklabels=True)
plt.show()

### Cross validation

In [ ]:
# Creating array of penalties

n_alpha = 100 # Number of penalties
min_alpha = .01 # min penalty
max_alpha = 200 # Maximum penalty
alpha_vals = np.logspace(np.log10(min_alpha),np.log10(max_alpha),n_alpha)
alpha_vals = alpha_vals[::-1] # reversing array (some sklearn standard?)

#### CV Lasso

In [ ]:
cv_lasso = linear_model.LassoCV(cv=10, random_state=0, fit_intercept=False, alphas=alpha_vals).fit(X, y)

#### CV Ridge

In [ ]:
cv_ridge = linear_model.ElasticNetCV(cv=10, random_state=0, l1_ratio=0, fit_intercept=False, alphas=alpha_vals).fit(X, y)

#### CV Elastic net (0.5 ratio)

In [ ]:
cv_elnet = linear_model.ElasticNetCV(cv=10, random_state=0, l1_ratio=0.5, fit_intercept=False, alphas=alpha_vals).fit(X, y)

In [ ]:
print('Lasso score: ', cv_lasso.score(X,y))
print('Ridge score: ', cv_ridge.score(X,y))
print('Elastic net score: ', cv_elnet.score(X,y))

In [ ]:
plot_CV_MSE(alpha_vals,cv_lasso.mse_path_,cv_lasso.alpha_, 'Lasso')
plot_CV_MSE(alpha_vals,cv_ridge.mse_path_,cv_ridge.alpha_, 'Ridge')
plot_CV_MSE(alpha_vals,cv_elnet.mse_path_,cv_elnet.alpha_, 'Elastic net')

### Cross validation parameters

In [ ]:
# Creating array of penalties
n_alpha = 100 # Number of penalties
min_alpha = .01 # min penalty
max_alpha = 200 # Maximum penalty
alpha_vals = np.logspace(np.log10(min_alpha),np.log10(max_alpha),n_alpha)
alpha_vals = alpha_vals[::-1] # reversing array (some sklearn standard?)

### Ordinary least squares

In [ ]:
reg_lstsq = linear_model.LinearRegression(fit_intercept=False)
beta_ls = reg_lstsq.fit(X,y).coef_

### Lasso

In [ ]:
reg = linear_model.Lasso(max_iter = 10000, fit_intercept = False)
beta_lasso, best_beta_lasso, cv_out_lasso, min_alpha_lasso = feature_selection_cv(X, y, alpha_vals, reg)
    
# Best model   
reg_lasso = linear_model.Lasso(alpha=min_alpha_lasso, max_iter = 10000, fit_intercept = False)

### Ridge

In [ ]:
reg = linear_model.ElasticNet(max_iter = 10000, l1_ratio=0, fit_intercept = False)

beta_ridge, best_beta_ridge, cv_out_ridge, min_alpha_ridge = feature_selection_cv(X, y, alpha_vals, reg)
        
# Best model
reg_ridge = linear_model.ElasticNet(alpha=min_alpha_ridge, max_iter = 10000, l1_ratio=0.0, fit_intercept = False)

### Elastic net

In [ ]:
reg = linear_model.ElasticNet(max_iter = 10000, l1_ratio=0.5, fit_intercept = False)
beta_elnet, best_beta_elnet, cv_out_elnet, min_alpha_elnet = feature_selection_cv(X, y, alpha_vals, reg)

# Best model
reg_elnet = linear_model.ElasticNet(alpha=min_alpha_elnet, max_iter = 10000, l1_ratio=0.5, fit_intercept = False)

### Group lasso

In [ ]:
#group_keys= {"T":-1,
#            "RH":-1,
#            "":-1}
group_keys= {"T":1,
            "RH":2,
            "":-1}

groups = []
for var_name in df_train.columns[1:]:
    for key, value in group_keys.items():
        if key in var_name:
            groups.append(value)
            break
# Group the rooms, and outside together
#groups=[-1, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 10, 10, 10, 10, -1, -1]
print(*zip(groups, df_train.columns[1:]))

reg = GroupLasso(groups=groups, frobenius_lipschitz=True, subsampling_scheme=1,
                 fit_intercept=False, random_state=0, supress_warning=True, n_iter=10000, tol=0.0001,
    )
group_reg = 0.05
beta_glasso, best_beta_glasso, cv_out_glasso, min_alpha_glasso = feature_selection_cv(X, y, alpha_vals, reg, 
                                                                                      group_reg=group_reg)

In [ ]:
# Best model
reg_gl = GroupLasso(
    groups=groups,
    group_reg=group_reg*min_alpha_glasso,
    l1_reg=(1-group_reg)*min_alpha_glasso,
    frobenius_lipschitz=True,
    subsampling_scheme=1,
    fit_intercept=False,
    random_state=0,
    supress_warning=True,
    n_iter=10000,
    tol=0.0001,
    )

### Plotting coefficients vs lambda

In [ ]:
plot_coefficients(beta_ridge, alpha_vals, min_alpha_ridge, name='Ridge')
plt.savefig('beta_ridge.png')
plot_coefficients(beta_lasso, alpha_vals, min_alpha_lasso, name='Lasso')
plt.savefig('beta_lasso.png')
plot_coefficients(beta_elnet, alpha_vals, min_alpha_elnet, name='Elastic net')
plt.savefig('beta_elnet.png')
plot_coefficients(beta_glasso, alpha_vals, min_alpha_glasso, name='Grouped lasso')
plt.savefig('beta_gl.png')

In [ ]:
plot_CV_MSE(alpha_vals, -np.array(cv_out_ridge), min_alpha_ridge, 'Ridge')
plot_CV_MSE(alpha_vals, -np.array(cv_out_lasso), min_alpha_lasso, 'Lasso')
plot_CV_MSE(alpha_vals, -np.array(cv_out_elnet), min_alpha_elnet, 'Elastic net')
plot_CV_MSE(alpha_vals, -np.array(cv_out_glasso), min_alpha_glasso, 'Grouped Lasso')

In [ ]:
data = {'Feature': list(df_train.columns[1:])}
df_results = pd.DataFrame(data)
df_results['Least squares'] = beta_ls.T
df_results['Lasso'] = beta_lasso[:,alpha_vals==min_alpha_lasso]
df_results['Ridge'] = beta_ridge[:,alpha_vals==min_alpha_ridge]
df_results['Elastic net'] = beta_elnet[:,alpha_vals==min_alpha_elnet]
df_results['Group Lasso'] = beta_glasso[:,alpha_vals==min_alpha_glasso]
np.round(df_results,2)


### Creating train score and MSE table

In [ ]:
regs = [reg_lstsq,reg_ridge,reg_lasso,reg_elnet,reg_gl]
labels = ['Least squares','Ridge','Lasso','Elastic net','Group lasso']
cv_ls = cross_val_score(reg_lstsq, X, y, cv=10,
                                 scoring='neg_mean_squared_error')
meanMSE = [-cv_ls.mean(),
    np.min(-np.array(cv_out_lasso).mean(axis=1)),
    np.min(-np.array(cv_out_ridge).mean(axis=1)),
    np.min(-np.array(cv_out_elnet).mean(axis=1)),
    np.min(-np.array(cv_out_glasso).mean(axis=1))
          ]

train_scores = []
for reg in regs:
    reg.fit(X,y)
    train_scores.append(reg.score(X,y))
    
Errors_table = pd.DataFrame()
Errors_table['Model'] = ['Least squares','Ridge','Lasso','Elastic net','Group lasso']
Errors_table['Train scores'] = np.round(train_scores,3)
Errors_table['Mean MSE'] = np.round(meanMSE,1)
Errors_table

### Testing with another month

In [ ]:
months = [3]
regs = [reg_lstsq,reg_ridge,reg_lasso,reg_elnet,reg_gl]
labels = ['Least squares','Ridge','Lasso','Elastic net','Group lasso']

fig, axs = plt.subplots(len(regs))
fig.set_figheight(25)
fig.set_figwidth(10)

df_test = df[np.isin(df.index.month,months)]
df_test = df_test.drop(['weekday', 'week status','month','NSM'], axis=1) # dropping the features used for filtering

y_t = np.array(df_test['Appliances']).reshape(-1,1)
X_t = np.array(df_test[df_test.columns[1:]])

i = 0
for reg in regs:
    # Testing data
    X_t, y_t = standardize(X_t,y_t)
    reg.fit(X,y)
    y_p = reg.predict(X_t)
    axs[i].plot([min(y_p),max(y_p)],[min(y_p),max(y_p)],'k-')
    axs[i].scatter(reg.predict(X_t),y_t)
    axs[i].grid()
    axs[i].set_title(labels[i], loc='left')
    print('Train score, '+labels[i]+': ',np.round(reg.score(X,y),3))
    print('Test score: '+labels[i]+': ',np.round(reg.score(X_t,y_t),3))
    i+=1
    
for ax in axs.flat:
    ax.set(xlabel='Predicted appliances', ylabel='True appliances')
    
plt.show()

### Bootstrap CV

In [ ]:
boot_samples = len(df_train) #100
boot_size = len(df_train) #100

reg_lasso = linear_model.Lasso(max_iter=10000, fit_intercept=False)
betas_lasso = bootstrap_loop(X, y, alpha_vals, reg_lasso, b=boot_size, N_samples=boot_samples)

In [ ]:
reg_ridge = linear_model.ElasticNet(alpha=min_alpha_ridge, max_iter = 10000, l1_ratio=0.0, fit_intercept = False)
betas_ridge = bootstrap_loop(X, y, alpha_vals, reg_ridge, b=boot_size, N_samples=boot_samples)

In [ ]:
reg_elnet = linear_model.ElasticNet(alpha=min_alpha_elnet, max_iter=10000, l1_ratio=0.5, fit_intercept = False)
betas_elnet = bootstrap_loop(X, y, alpha_vals, reg_elnet, b=boot_size, N_samples=boot_samples)

In [ ]:
reg_gl = reg_gl = GroupLasso(
    groups=groups,
    group_reg=group_reg*min_alpha_glasso,#alpha_vals[i],
    l1_reg=(1-group_reg)*min_alpha_glasso,
    frobenius_lipschitz=True,
    #scale_reg="inverse_group_size",
    #scale_reg="inverse_group_size",
    subsampling_scheme=1,
    fit_intercept=False,
    random_state=0,
    supress_warning=True,
    n_iter=10000,
    tol=0.0001,
    )
betas_gl = bootstrap_loop(X, y, alpha_vals, reg_gl, b=boot_size, N_samples=boot_samples, group_reg=group_reg)

In [ ]:
plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')
plt.boxplot(betas_lasso,labels=df_train.columns[1:])
plt.xticks(rotation='vertical')
plt.title('Lasso')
plt.ylabel('Coefficients')
plt.savefig('boxplot_lasso.pdf')
plt.show()

plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')
plt.boxplot(betas_ridge,labels=df_train.columns[1:])
plt.xticks(rotation='vertical')
plt.title('Ridge')
plt.ylabel('Coefficients')
plt.show()

plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')
plt.boxplot(betas_elnet,labels=df_train.columns[1:])
plt.xticks(rotation='vertical')
plt.title('Elastic Net')
plt.ylabel('Coefficients')
plt.show()


plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')
plt.boxplot(betas_gl,labels=df_train.columns[1:])
plt.xticks(rotation='vertical')
plt.title('Group Lasso')
plt.ylabel('Coefficients')
plt.show()


In [ ]:
number_of_zeros = np.sum(betas_lasso == 0,axis=0)
y_pos = np.arange(len(df_train.columns[1:]))

# Sorting in descenting order
labels = df_train.columns[1:][number_of_zeros.argsort()][::-1]
number_of_zeros[::-1].sort()

plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')
plt.barh(y_pos,number_of_zeros ,align='center', alpha=0.5)
plt.yticks(y_pos, labels)
plt.xlabel('Number of times coefficient is zero')
plt.savefig('barchart_lasso.pdf')
plt.show()
